In [1]:
from dash import html, dcc, Dash
from jupyter_dash import JupyterDash
from dash.dependencies import Input, Output
from dash.exceptions import PreventUpdate
import plotly.express as px
import pandas as pd
import seaborn as sns
from dash import html, dcc
import dash_bootstrap_components as dbc

In [2]:
df = pd.read_csv('dataset.csv')
df.head(5)

,HealthTopic,Population,Indicator,Unit,Time,RegionCode,RegionName,NumValue,TxtValue
0,Leptospirosis,Confirmed cases,Reported cases,N,2007,AT,Austria,9.0,NaN
1,Leptospirosis,Confirmed cases,Reported cases,N,2007,BE,Belgium,8.0,NaN
2,Leptospirosis,Confirmed cases,Reported cases,N,2007,BG,Bulgaria,16.0,NaN
3,Leptospirosis,Confirmed cases,Reported cases,N,2007,CY,Cyprus,0.0,NaN
4,Leptospirosis,Confirmed cases,Reported cases,N,2007,CZ,Czechia,24.0,NaN


In [19]:
#!pip install pycountry
import pycountry


def convert_alpha_2_to_3(alpha_2_code):
    try:
        return pycountry.countries.get(alpha_2=alpha_2_code).alpha_3
    except:
        return None  

df['RegionCodeAlpha3'] = df['RegionCode'].apply(convert_alpha_2_to_3)
df = df.dropna(subset=['RegionCodeAlpha3'])

In [30]:
# Initialize app with a Bootstrap theme for dark mode
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.SLATE])

# Define the layout of the app
app.layout = dbc.Container(
    [
        dbc.Row(
            dbc.Col(
                html.H3("Group-02-Final-Dashboard", className="text-center text-light mt-4"),
                width=12
            )
        ),
        dbc.Row(
            dbc.Col(
                html.Div(
                    [
                        "Select a country to analyze: ",
                        dcc.Dropdown(
                            id="state_dropdown",
                            options=[{'label': state, 'value': state} for state in df['RegionName'].unique()],
                            value="Austria",  # Default value
                            className="mb-4",
                            style={
                                'background-color': '#2b2b2b',  # Dark background for dropdown
                                'color': '#ffffff',  # White text for selected value
                                'width': '100%'  # Set dropdown width
                            }
                        ),
                    ],
                    className="text-light"
                ),
                width=12
            )
        ),
        dbc.Row(
            dbc.Col(
                html.Div(
                    [
                        "Filter by RegionCode: ",
                        dcc.Dropdown(
                            id="regioncode_dropdown",
                            options=[{'label': code, 'value': code} for code in df['RegionCodeAlpha3'].unique()],
                            value="AUT",  # Default value (Austria RegionCode)
                            className="mb-4",
                            style={
                                'background-color': '#2b2b2b',  # Dark background for dropdown
                                'color': '#ffffff',  # White text for selected value
                                'width': '100%'  # Set dropdown width
                            }
                        ),
                    ],
                    className="text-light"
                ),
                width=12
            )
        ),
        dbc.Row(
            [
                dbc.Col(dcc.Graph(id="visual"), width=6),  # Line graph
                dbc.Col(dcc.Graph(id="world_map"), width=6)  # World map
            ]
        ),
        dbc.Row(
            [
                dbc.Col(dcc.Graph(id="bar_chart"), width=6),  # Bar chart
                dbc.Col(dcc.Graph(id="donut_plot"), width=6)  # Donut plot
            ]
        )
    ],
    fluid=True
)

# Callback to update all plots based on the dropdown selections
@app.callback(
    [
        Output("visual", "figure"), 
        Output("world_map", "figure"), 
        Output("bar_chart", "figure"),
        Output("donut_plot", "figure")
    ],
    [
        Input("state_dropdown", "value"), 
        Input("regioncode_dropdown", "value")
    ]
)
def update_output_div(state, region_code):
    if not state or not region_code:
        raise PreventUpdate

    # Filter data based on selected state
    filtered_df = df.query(f"RegionName == '{state}'")

    # Line chart for total leptospirosis cases over time
    line_fig = px.line(
        filtered_df,
        x="Time",
        y="NumValue",
        title=f"Confirmed Leptospirosis Cases Over Time in {state}",
        template="plotly_dark",  # Dark background for plot
        labels={"NumValue": "Confirmed Leptospirosis Cases"}
    )

    # World map to show global overview
    map_fig = px.choropleth(
        df,
        locations="RegionCodeAlpha3",  # Using ISO alpha-3 country codes
        color="NumValue",              # Color shading based on NumValue
        hover_name="RegionName",        # Hover shows country names
        title="Global Overview of Confirmed Leptospirosis Cases",
        template="plotly_dark",         # Dark background for the map
        color_continuous_scale=px.colors.sequential.Plasma,  # Color gradient scale
        labels={'NumValue': 'Confirmed Cases'},  # Legend label
        projection="natural earth"  # World map projection
    )

    # Bar chart filtered by RegionCode, x-axis: Time, y-axis: NumValue
    filtered_region_df = df.query(f"RegionCodeAlpha3 == '{region_code}'")
    bar_fig = px.bar(
        filtered_region_df,
        x="Time",
        y="NumValue",
        title=f"Leptospirosis Cases Over Time in RegionCode {region_code}",
        template="plotly_dark",  # Dark background for the plot
        labels={"NumValue": "Confirmed Leptospirosis Cases"}
    )

    # Donut plot showing top 5 RegionCodes with highest NumValue counts
    top_5_region_df = df.groupby('RegionCodeAlpha3').sum().nlargest(5, 'NumValue').reset_index()
    donut_fig = px.pie(
        top_5_region_df,
        names="RegionCodeAlpha3",
        values="NumValue",
        title="Top 5 RegionCodes by Confirmed Leptospirosis Cases",
        template="plotly_dark",  # Dark background for the plot
        hole=0.4,  # Donut hole size
        labels={"NumValue": "Confirmed Cases"}
    )

    # Update donut plot layout for better appearance
    donut_fig.update_traces(textposition='inside', textinfo='percent+label')

    # Adjust color range if needed (optional)
    map_fig.update_layout(
        coloraxis_colorbar=dict(
            title="Confirmed Cases",
            ticks="outside"
        )
    )

    return line_fig, map_fig, bar_fig, donut_fig

# Run the server
if __name__ == "__main__":
    app.run_server(debug=True, port=4819)


c:\Users\Tharindu\anaconda3\Lib\site-packages\dash\dash.py:550: UserWarning:

JupyterDash is deprecated, use Dash instead.
See https://dash.plotly.com/dash-in-jupyter for more details.



Dash app running on http://127.0.0.1:4819/
